# Run NVIDIA Sionna on a Chameleon server

[NVIDIA Sionna™](https://developer.nvidia.com/sionna) is a GPU-accelerated open-source library for link-level simulations. It enables rapid prototyping of complex communication system architectures and provides native support for the integration of machine learning in 6G signal processing.

This notebook will help you get a Sionna container running in a GPU instance on Chameleon.


## Provision the resource


In [ ]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import hardware

username = os.getenv('USER') # all exp resources will have this prefix

### Check resource availability

This notebook will try to reserve a bare metal Ubuntu server with RTX6000 GPU on CHI@UC - pending availability. Before you begin, you should check the host calendar at [https://chi.uc.chameleoncloud.org/project/leases/calendar/host/](https://chi.uc.chameleoncloud.org/project/leases/calendar/host/). In the "Node Type" dropdown, filter on `gpu_rtx_6000` and make sure some hosts are available.

Alternatively, you can use a different NVIDIA GPU type at CHI@UC or CHI@TACC.

### Chameleon configuration

You can change your Chameleon project name (if not using the one that is automatically configured in the JupyterHub environment) and the site on which to reserve resources (depending on availability) in the following cell.

In [ ]:
context.version = "1.0" # required during transition
context.choose_site(default="CHI@UC")
context.choose_project()


If you need to change the details of the Chameleon server, e.g. use a different GPU type depending on availability, you can do that in the following cell.

In [ ]:
node_type = "gpu_rtx_6000"

Some GPU instance types are:

* CHI@UC: `gpu_rtx_6000` (1x RTX6000), `gpu_v100` (4x V100), `compute_gigaio` (1x A100)
* CHI@TACC: `compute_liqid` (1x A100), `gpu_p100` (2x P100), `compute_gigaio` (1x A100 or 1x A30 on some nodes, but no GPU on some other nodes)

(you can check what is available following the instructions in the next cell.)

### Reservation

The following cells will create a reservation that begins now, and ends in 8 hours, *if* your requested node type is available.

If the node type you have requested is *not* available right now, it will start your reservation as soon as one is available.

You can refer to the [CHI@UC host calendar](https://chi.uc.chameleoncloud.org/project/leases/calendar/host/) to see availability (change the "Node type" selection to `gpu_rtx_6000`), or refer to the [CHI@TACC host calendar](https://chi.tacc.chameleoncloud.org/project/leases/calendar/host/) to see availability for other node types with GPU.

In [ ]:
gpu_start_times = [n.next_free_timeslot()[0] for n in hardware.get_nodes(node_type=node_type)]
current_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(minutes=1)
if min(gpu_start_times) > current_time:
    lease_start = min(gpu_start_times)
    print(f"There is no {node_type} available now, you can request one starting at {str(lease_start)} (UTC).")
else:
    lease_start = current_time
    print(f"A {node_type} IS available now, your lease will start right away.")


In [ ]:
l = lease.Lease(f"sionna-{username}", duration=datetime.timedelta(hours=8),
                start_date = lease_start  )
l.add_node_reservation(node_type = node_type, amount = 1) 
l.add_fip_reservation(1) 
l.submit(idempotent=True)

### Provisioning resources

This section provisions resources. It will take approximately 10 minutes. You can check on its status in the Chameleon web-based UI: [https://chi.uc.chameleoncloud.org/project/instances/](https://chi.uc.chameleoncloud.org/project/instances/), then come back here when it is in the READY state.

In [ ]:
# continue here, whether using a lease created just now or one created earlier
l = lease.get_lease(f"sionna-{username}")

In [ ]:
s = server.Server(
    f"sionna-{username}", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

Associate an IP address with this server:

In [ ]:
reserved_fip = l.get_reserved_floating_ips()[0]
s.associate_floating_ip(reserved_fip)

and wait for it to come up:

In [ ]:
s.check_connectivity(host=reserved_fip)

## Log in to resource

To log in to the resource, use File > New > Terminal in the Chameleon JupyterHub environment, or your local terminal, and paste in the *output* of the following cell:


In [ ]:
print("ssh cc@" + reserved_fip)

## Set up Docker

To use common deep learning frameworks like Tensorflow or PyTorch, we can run *containers* that have all the prerequisite libraries necessary for these frameworks. Here, we will set up the container framework.

In [ ]:
node.run("curl -sSL https://get.docker.com/ | sudo sh")
node.run("sudo groupadd -f docker; sudo usermod -aG docker $USER")

In [ ]:
node = ssh.Remote(reserved_fip) # note: need a new SSH session to get new group membership
node.run("docker run hello-world")

In [ ]:
# get NVIDIA container toolkit 
node.run("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
node.run("sudo apt update")
node.run("sudo apt-get install -y nvidia-container-toolkit")
node.run("sudo nvidia-ctk runtime configure --runtime=docker")
node.run("sudo systemctl restart docker")

In [ ]:
# check that we can see GPU from inside container
node.run("docker run --rm --gpus all ubuntu nvidia-smi")

## Build the Sionna container

In [ ]:
node.run("git clone https://github.com/NVlabs/sionna")

In [ ]:
node.run("cd sionna; make docker") # this will take a while

In [ ]:
node.run("cd sionna; make run-docker gpus=all")

## Connect to your Sionna container

Run the following cell:

In [ ]:
print('ssh -L 127.0.0.1:8888:127.0.0.1:8888 cc@' + reserved_fip) 

and copy the SSH command that is printed. In a **local terminal on your own laptop**, run the SSH command that is printed by the previous cell. This will set up a tunnel to the Jupyter server that you are running on a Chameleon instance.

If your Chameleon key is not in the default location, you should also specify the path to your key as an argument, using `-i`. For example:

In [ ]:
print('ssh -i ~/.ssh/id_rsa_chameleon -L 127.0.0.1:8888:127.0.0.1:8888 cc@' + reserved_fip) 

Leave this SSH session open.

Now, you can open the following URL in a browser:

http://127.0.0.1:8888/lab

and run the example notebooks shown there.

## Release resources

If you finish with your experimentation before your lease expires,release your resources and tear down your environment by running the following (commented out to prevent accidental deletions).

This section is designed to work as a "standalone" portion - you can come back to this notebook, ignore the top part, and just run this section to delete your reasources.

In [ ]:
# setup environment - if you made any changes in the top part, make the same changes here
import chi, os
from chi import lease, magic, context

context.version = "1.0" # required during transition

context.choose_site(default="CHI@UC")
context.choose_project()
username = os.getenv('USER') # all exp resources will have this prefix

l = lease.get_lease(f"sionna-{username}")

In [ ]:
# un-comment to free resources
# chi.magic.cleanup_resources(l.id)
